In [1]:
import numpy as np
import pandas as pd

In [2]:
df2=pd.read_spss("Desktop/good.sav")

In [3]:
for col in df2.columns:
    if df2[col].isna().any():
        print(col,df2[col].dtype)

In [4]:
df2['product'].value_counts()

Benzodiazepines                      68536
Diphenhydramine                      51798
Acetaminophen                        38033
Acetylsalicylic Acid Alone           15268
Bupropion                            12328
Lithium                               9434
Calcium Antagonist, Alone             2945
Methadone                             2791
Oral Hypoglycemics: Sulfonylureas     2689
Oral Hypoglycemics: Biguanides        2205
Name: product, dtype: int64

In [5]:
df2.shape

(206027, 134)

In [6]:
from keras.models import Sequential
from keras.layers import Dense

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X = df2.drop('product', axis = 1)
y = df2['product']

from sklearn.model_selection import cross_val_score


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)




In [8]:
model = Sequential()
model.add(Dense(input_dim = 133, units = 64, activation='relu'))
model.add(Dense(units = 32, activation='relu'))
model.add(Dense(units = 32, activation='tanh'))
model.add(Dense(units = 32, activation='sigmoid'))
model.add(Dense(units = 1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                8576      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 12,801
Trainable params: 12,801
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='categorical_crossentropy', optimizer='Ftrl', metrics=['accuracy'])

In [10]:
y_train=pd.get_dummies(y_train,drop_first=True).iloc[:,0].values
y_test=pd.get_dummies(y_test,drop_first=True).iloc[:,0].values

In [11]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_test,y_test),batch_size=64)

Epoch 1/50
2157/2157 [==============================] - 8s 3ms/step - loss: 0.0000e+00 - accuracy: 0.9254 - val_loss: 0.0000e+00 - val_accuracy: 0.9259
Epoch 2/50
2157/2157 [==============================] - 5s 2ms/step - loss: 0.0000e+00 - accuracy: 0.9259 - val_loss: 0.0000e+00 - val_accuracy: 0.9259
Epoch 3/50
2157/2157 [==============================] - 4s 2ms/step - loss: 0.0000e+00 - accuracy: 0.9248 - val_loss: 0.0000e+00 - val_accuracy: 0.9259
Epoch 4/50
2157/2157 [==============================] - 5s 2ms/step - loss: 0.0000e+00 - accuracy: 0.9249 - val_loss: 0.0000e+00 - val_accuracy: 0.9259
Epoch 5/50
2157/2157 [==============================] - 5s 2ms/step - loss: 0.0000e+00 - accuracy: 0.9264 - val_loss: 0.0000e+00 - val_accuracy: 0.9259
Epoch 6/50
2157/2157 [==============================] - 5s 2ms/step - loss: 0.0000e+00 - accuracy: 0.9257 - val_loss: 0.0000e+00 - val_accuracy: 0.9259
Epoch 7/50
2157/2157 [==============================] - 5s 2ms/step - loss: 0.0000e+00 -

In [12]:
from sklearn.model_selection import GridSearchCV

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
xgb_model= xgb.XGBClassifier(learning_rate= 0.1,max_depth= 5, min_child_weight= 4,n_estimators= 100)
X = df2.drop('product', axis = 1)
y = df2['product']
sc = StandardScaler()
sc.fit(X)
X2 = sc.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.33, random_state=42)

# cls.fit(X_train,y_train)
xgb_model.fit(X_train,
        y_train,
        verbose=True,
        early_stopping_rounds=10,
        eval_metric='aucpr',
        eval_set=[(X_test,y_test)])
       
y_pred = xgb_model.predict(X_train)
print(accuracy_score(y_train, y_pred))
y_pred = xgb_model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
import matplotlib.pyplot as plt  
from sklearn.datasets import make_classification
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(xgb_model, 
                      X_test,
                      y_test)
                      
plt.show()
metrics.plot_roc_curve(xgb_model, X_test, y_test)  
plt.show() 

In [ ]:
def xgb_grid_search(X,y,nfolds):
    #create a dictionary of all values we want to test
    param_grid = { 'max_depth': [4,5,6,15] , 'min_child_weight':[4,5,6] ,'learning_rate': [0.05,0.1,0.5] ,'n_estimators': [20,50,100] }
    # decision tree model
    xgb_model=XGBClassifier()
    #use gridsearch to test all values
    xgb_gscv = GridSearchCV(xgb_model, param_grid, cv=nfolds)
    #fit model to data
    xgb_gscv.fit(X, y)
    print(xgb_gscv.best_params_)
    print(xgb_gscv.best_estimator_)
    print(xgb_gscv.best_score_)

In [ ]:
# import xgboost as xgb
# from sklearn.metrics import accuracy_score
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split

# parameters={"loss": ["deviance", "exponential"], 'learning_rate':[0.0001,0.001,0.01],'n_estimators':[100,500,30],
#            "min_samples_split":[2,4,6,8,10],"max_depth":[2,5,8,10,13,15,20,25,30],"max_features":["auto", "sqrt","log2"]}

# cls = xgb.XGBClassifier()
# grid_search=GridSearchCV(cls,param_grid=parameters,cv=5)
# grid_search.fit(X_train, y_train)


# X = df2.drop(['product'], axis = 1)
# y = df2['product']
# # sc = StandardScaler()
# # sc.fit(X)
# # X2 = sc.transform(X)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)



In [ ]:

xgb_model= xgb.XGBClassifier()
grid_search=GridSearchCV(xgb_model,param_grid=parameters,cv=5)
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
best_model=grid_search.best_estimator_
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_train)
print(accuracy_score(y_train, y_pred))
y_pred = best_model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
# cls.fit(X_train,y_train)
cls.fit(X_train,
        y_train)
       
y_pred = cls.predict(X_train)
print(accuracy_score(y_train, y_pred))
y_pred = cls.predict(X_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
from msdlib import msd
score,conf = msd.class_result(y_test, y_pred,out_confus = True)

In [ ]:
score

In [ ]:
import matplotlib.pyplot as plt  
from sklearn import datasets, metrics, model_selection, svm
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
predictions = cls.predict(X_test)
cm = confusion_matrix(y_test, predictions, labels=cls.classes_) 
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
predictions = cls.predict(X_test)
cm = confusion_matrix(y_test, predictions, labels=cls.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=cls.classes_)
disp.plot() 

In [ ]:
conf

In [ ]:

parameters={"loss": ["deviance", "exponential"], 'learning_rate'[0.0001,0.001,0.01],'n_estimators':[100,500,30],
           "min_samples_split":[2,10,2],"max_depth":[2,30,2],"max_features":[‘auto’, ‘sqrt’, ‘log2’]}

cls = xgb.XGBClassifier()
grid_search=GridSearchCV(cls,param_grid=parameters,cv=5)
grid_search.fit(X_train, y_train)